In [1]:
import graphlab as gl
gl.canvas.set_target('ipynb')

In [2]:
#adapted from jschaub30 github
#https://gist.github.com/jschaub30/54054f81508003d01d5d

import sys
import re

mygml = 'data/power.gml'

with open(mygml, 'r') as f:
        blob = f.read()
blob = ''.join(blob.split('node')[1:])
nodes = blob.split('edge')[0]
edges = ''.join(blob.split('edge')[1:]).strip().rstrip(']')

pattern = re.compile(r'(\d+)') 
node_list = re.findall(pattern, nodes)
nodes = gl.SFrame({'id': [int(i) for i in node_list] })

pattern = re.compile(r'(\d+)') 
edge_list = re.findall(pattern, edges)
edges = gl.SFrame({'src': [int(i) for i in edge_list[::2]], 
                   'dst': [int(i) for i in edge_list[1::2]]})

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1497756845.log


This non-commercial license of GraphLab Create for academic use is assigned to nicholas.capofari@spsmail.cuny.edu and will expire on June 06, 2018.


In [3]:
g = gl.SGraph()

g = g.add_vertices(nodes, vid_field='id')
g = g.add_edges(edges=edges, 
                src_field='src', 
                dst_field='dst')

In [4]:
#take a look at total degree 
m = gl.degree_counting.create(g)
v = m['graph'].vertices
subgraph = g.get_neighborhood(v[v['total_degree'].argmax()]['__id'])
subgraph.show(vlabel='id')

In [5]:
#calculate degree centrality
from __future__ import division

v['d_centrality'] = v['total_degree'].apply(lambda x: (x/(len(v['__id'])-1)))
v = v.sort('d_centrality', ascending=False)
v.head()

__id,in_degree,out_degree,total_degree,d_centrality
2553,19,0,19,0.00384615384615
4458,5,13,18,0.00364372469636
3468,8,6,14,0.00283400809717
831,14,0,14,0.00283400809717
4345,11,3,14,0.00283400809717
2542,11,2,13,0.00263157894737
2575,11,2,13,0.00263157894737
2382,6,7,13,0.00263157894737
3895,4,9,13,0.00263157894737
2585,13,0,13,0.00263157894737


In [6]:
#to find undirected shortest paths add edges to the SGraph in both directions
g = g.add_edges(edges=g.edges, 
                src_field='__dst_id', 
                dst_field='__src_id')

#takes too long to find diameter if we use every node
#take top 50 nodes in terms of degree centrality
longest_sp = []
#calculate sum of sps to calculate closeness centrality
cc = []

#shortest path for each
for each in v['__id'][:50]:
    sp = gl.shortest_path.create(g, each, verbose=False)
    temp = sp['distance']['distance']
    longest_sp.extend([temp.max()])
    cc.extend([(len(v['__id'])-1)/sum(temp)])

diameter = max(longest_sp)
diameter

44.0

In [7]:
top_50 = v[0:50]
top_50['c_centrality'] = cc
top_50.sort('c_centrality', ascending=False).head()

__id,in_degree,out_degree,total_degree,d_centrality,c_centrality
490,11,0,11,0.00222672064777,0.0727444079577
2608,6,4,10,0.00202429149798,0.0707574195027
2617,5,7,12,0.00242914979757,0.0690117627336
1030,5,5,10,0.00202429149798,0.0681529716903
2542,11,2,13,0.00263157894737,0.0680253373726
1106,9,1,10,0.00202429149798,0.0679813395351
1166,6,4,10,0.00202429149798,0.0677863768593
4458,5,13,18,0.00364372469636,0.0675453948808
2586,2,8,10,0.00202429149798,0.0673869154799
2282,6,5,11,0.00222672064777,0.067135984344


In [8]:
subgraph = g.get_neighborhood(top_50['__id'][:3])
subgraph.show(vlabel='__id', highlight=top_50['__id'])

In [9]:
#pagerank of each node
pr = gl.pagerank.create(g, max_iterations=10, verbose=False)
pr_out = pr['pagerank']
top_pr = pr_out.topk('pagerank', k=5)
print top_pr

+------+---------------+-----------------+
| __id |    pagerank   |      delta      |
+------+---------------+-----------------+
| 4458 | 5.98539719053 | 0.0548204559701 |
| 3468 | 5.02606554054 |  0.423393855768 |
| 831  | 4.94197061655 |  0.694933987117 |
| 2553 | 4.93120134233 | 0.0245761875212 |
| 1224 | 4.46150148994 |  0.376075521721 |
+------+---------------+-----------------+
[5 rows x 3 columns]



In [10]:
subgraph = g.get_neighborhood(top_pr['__id'])
subgraph.show(vlabel='__id', highlight=top_pr['__id'])